In [1]:
from SoapySDR import *
import SoapySDR
import pyaudio
import signal
import queue
import cupy as cp
import cusignal as sig

#### Demodulator Settings

In [2]:
fm_freq = 96.9e6 # FM Station Frequency
samp_rate = int(240e3)
audio_fs = int(48e3)
buff_len = int(1024*(samp_rate//audio_fs))

#### SoapySDR Configuration

In [3]:
args = dict(driver="lime") # SDR Selection (eg.: lime, rtlsdr, plutosdr...)
sdr = SoapySDR.Device(args)
sdr.setSampleRate(SOAPY_SDR_RX, 0, samp_rate)
sdr.setFrequency(SOAPY_SDR_RX, 0, fm_freq)

#### FIFO Buffer and Shared Memory Allocation

In [4]:
que = queue.Queue()
buff = sig.get_shared_mem(buff_len, dtype=cp.complex64)

#### PyAudio Asynchronous Buffer

In [5]:
def demod(in_data, frame_count, time_info, status):
    b = cp.array(que.get())
    b = cp.diff(cp.unwrap(cp.angle(b)))
    b = sig.resample_poly(b, 1, 5, window='hamm')
    b /= cp.pi
    b = cp.asnumpy(b).astype(cp.float32)
    return (b, pyaudio.paContinue)

p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paFloat32, channels=1, rate=audio_fs, output=True, stream_callback=demod)

#### Graceful Exit Handler

In [6]:
def signal_handler(signum, frame):
    stream.stop_stream()
    stream.close()
    p.terminate()
    sdr.closeStream(rx)
    exit(-1)
    
signal.signal(signal.SIGINT, signal_handler)

<function _signal.default_int_handler>

#### Start Collecting Data

In [ ]:
rx = sdr.setupStream(SOAPY_SDR_RX, SOAPY_SDR_CF32)
sdr.activateStream(rx)
stream.start_stream()

while True:
    sdr.readStream(rx, [buff], buff_len, timeoutUs=int(8e12))
    que.put(buff.astype(cp.complex64).copy())